In [87]:
from flask import Flask, request, jsonify, render_template,url_for
import sqlite3
import json
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score


In [88]:
process_db= sqlite3.connect('process_values.db')
cursor1 = process_db.cursor()

In [95]:
# build linear regression

type(process_db)


sqlite3.Connection

In [96]:
db_data = cursor1.execute("SELECT * FROM sensor_data").fetchall()
sensor_df = pd.DataFrame(db_data,columns= ['sensor1','sensor2'])

In [97]:
sensor_df


,sensor1,sensor2
0,5.85591527310501,38.607495226349
1,4.55997309445233,9.067653208189501
2,2.046294817515498,-9.756507888205919
3,6.518983822839673,52.1576676454831
4,1.6533208155938561,-7.610798226609832
...,...,...
197,2.98425804040893,-7.082319955729961
198,0.6313011524076773,-6.143580839554269
199,6.1798896618546095,43.67250809494191
200,-3.33,"b'\x02\rMA\xd4""H\xc0'"


In [98]:
sensor1 = sensor_df['sensor1']
sensor2 = sensor_df['sensor2']

In [99]:
sensor1

0        5.85591527310501
1        4.55997309445233
2       2.046294817515498
3       6.518983822839673
4      1.6533208155938561
              ...        
197      2.98425804040893
198    0.6313011524076773
199    6.1798896618546095
200                 -3.33
201                  5.99
Name: sensor1, Length: 202, dtype: object

In [94]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly2 = PolynomialFeatures(degree = 2, include_bias = False)
poly3 = PolynomialFeatures(degree = 3, include_bias = False)

poly2_sens1 = poly2.fit_transform(sensor1.values.reshape(-1,1))
poly3_sens1 = poly3.fit_transform(sensor1.values.reshape(-1,1))


In [67]:
poly2_df_og = pd.DataFrame(poly2_sens1)
poly3_df_og = pd.DataFrame(poly3_sens1)

In [69]:
poly2_df_og

,0,1
0,5.855915,34.291744
1,4.559973,20.793355
2,2.046295,4.187322
3,6.518984,42.497150
4,1.653321,2.733470
...,...,...
195,1.600921,2.562948
196,6.585706,43.371523
197,2.984258,8.905796
198,0.631301,0.398541


In [77]:
linreg_poly2 = LinearRegression().fit(poly2_df_og, sensor2)
linreg_poly3 = LinearRegression().fit(poly3_df_og, sensor2)

In [78]:
pickle.dump(linreg_poly2, open('linreg_poly2','wb'))
pickle.dump(linreg_poly3, open('linreg_poly3','wb'))



In [79]:
value = '3.333'

In [80]:
list1 = pd.Series(float(value))
type(list1)

pandas.core.series.Series

In [81]:
thing = list1.values.reshape(-1,1)

In [82]:
poly2_sens1 = poly2.fit_transform(thing)
poly3_sens1 = poly3.fit_transform(thing)



In [83]:
poly2_df = pd.DataFrame(poly2_sens1)
poly3_df = pd.DataFrame(poly3_sens1)
    
model_poly2 = pickle.load(open('linreg_poly2', 'rb'))
model_poly3 = pickle.load(open('linreg_poly3', 'rb'))

result_poly2 = model_poly2.predict(poly2_df)
result_poly3 = model_poly3.predict(poly3_df)

print(result_poly2, result_poly3)

[6.43285953] [-6.3910873]


In [101]:
type(result_poly3[0])

numpy.float64

In [84]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_pred_poly2 = linreg_poly2.predict(poly2_df_og)
y_pred_poly3 = linreg_poly3.predict(poly3_df_og)


In [86]:

mse_2 = mean_squared_error(sensor2, y_pred_poly2)
mse_3 = mean_squared_error(sensor2, y_pred_poly3)

rmse_2 = sqrt(mse_2)
rmse_3 = sqrt(mse_3)

print(rmse_2,rmse_3)

20.463233328335804 5.140013122602249
